### Part 3: Predicting hypertension
#### Part a:

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.metrics import roc_curve
from nltk.tokenize import word_tokenize

In [35]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [97]:
class Data:
    def __init__(self, data, target):
        self.data = data
        self.target = target
    
class Dataset:
    def __init__(self, train: Data, test: Data):
        self.train = train
        self.test = test

In [133]:
#pt = pd.DataFrame([[1,0,1],[2,0,1],[3,1,0]], columns=["id","label","train"])
#cts = pd.DataFrame([[1,1,5],[1,1,10],[1,2,6],[2,1,4],[3,1,20],[3,1,30],[3,1,40]], columns=["id","type","hr"])

def prepare_data(patients: pd.DataFrame, charts: pd.DataFrame, itemid: int) -> Dataset:
    df = charts
    df = df[df.itemid == itemid].drop("itemid", axis=1) \
                                .groupby("hadm_id") \
                                .filter(lambda x: len(x) >= 2) \
                                .groupby("hadm_id") \
                                .agg([np.min, np.max, np.mean]) \
                                .reset_index()
    #df.columns = [''.join(col) if col[1] == '' else '_'.join(col) for col in df.columns.values]
    df.columns = [''.join(col) if col[1] == '' else col[1] for col in df.columns.values]
    df = pd.merge(df, patients, on="hadm_id")

    #min_max_scaler = MinMaxScaler()
    #df.iloc[:,1:4] = min_max_scaler.fit_transform(df.iloc[:,1:4].values.astype(float))

    trainset = df[df.train == 1].copy()
    trainset.reset_index(drop=True, inplace=True)
    trainset.drop(["train", "hadm_id"], axis=1, inplace=True)
    target = trainset.pop("hypertension")
    train = Data(trainset, target)

    testset = df[df.train == 0].copy()
    testset.reset_index(drop=True, inplace=True)
    testset.drop(["train", "hadm_id"], axis=1, inplace=True)
    target = testset.pop("hypertension")
    test = Data(testset, target)

    return Dataset(train, test)

In [128]:
patients_dataset = pd.read_csv("~/csc2548_ml4h/hypertension_patients.gz", compression='gzip')
charts_dataset = pd.read_csv("~/csc2548_ml4h/hypertension_charts.gz", compression='gzip')

patients_dataset.drop("subject_id", axis=1, inplace=True)
charts_dataset.drop("subject_id", axis=1, inplace=True)

In [134]:
#patients_dataset.head()
#charts_dataset.head()
#patients_dataset[patients_dataset.hadm_id.duplicated()].head()
# patients_dataset[patients_dataset.subject_id == 6953]
hr_data = prepare_data(patients_dataset, charts_dataset, 220045)
resp_data = prepare_data(patients_dataset, charts_dataset, 220210)
o2_data = prepare_data(patients_dataset, charts_dataset, 220277)
bp_data = prepare_data(patients_dataset, charts_dataset, 220181)

In [135]:
hr_data.train.data.head()
hr_data.train.target.head()
hr_data.test.data.head()
hr_data.test.target.head()

,amin,amax,mean
0,75.0,131.0,111.313953
1,69.0,104.0,81.218182
2,46.0,92.0,68.968750
3,56.0,109.0,80.135135
4,68.0,142.0,95.351536


0    0
1    1
2    1
3    0
4    0
Name: hypertension, dtype: int64

,amin,amax,mean
0,64.0,117.0,79.382353
1,72.0,97.0,84.307692
2,73.0,162.0,93.312925
3,62.0,115.0,73.945946
4,72.0,90.0,80.972222


0    0
1    1
2    1
3    1
4    1
Name: hypertension, dtype: int64